# Network and training VGG inspired net

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import os 
import h5py

plt.rc('image', cmap='gist_gray')

import tensorflow as tf

from sklearn.metrics import recall_score
from sklearn.utils import shuffle

import keras
from keras.utils import Sequence
from keras.applications.vgg16 import VGG16
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import keras.backend as K

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Activation

from classification_models import Classifiers

import time 

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
tf.__version__

In [ ]:
print('tf', tf.__version__)
print('keras', keras.__version__)

In [ ]:
path = '/path/to/working/dir'
data_dir = os.path.join(path, 'data_folder')
result_dir = os.path.join(path, 'results_folder')

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "6"
set_session(tf.Session(config=config))

## Load + preprocess data

In [ ]:
start_time = time.time()

In [ ]:
train_h5 = h5py.File(os.path.join(data_dir, 'training_data.h5'), 'r')
val_h5 = h5py.File(os.path.join(data_dir, 'val_data.h5'), 'r')
holdout_h5 = h5py.File(os.path.join(data_dir, 'holdout_data.h5'), 'r')

In [ ]:
X_train, y_train_binary = train_h5['X'], train_h5['y']
X_val, y_val_binary = val_h5['X'], val_h5['y']
X_holdout, y_holdout_binary = holdout_h5['X'], holdout_h5['y']

In [ ]:
print(len(X_train), len(X_val), len(X_holdout))

In [ ]:
y_train = keras.utils.to_categorical(y_train_binary, num_classes=2)[:42000]
y_val = keras.utils.to_categorical(y_val_binary, num_classes=2)[:6000]
y_holdout = keras.utils.to_categorical(y_holdout_binary, num_classes=2)[:12000]

In [ ]:
fig = plt.figure(figsize = (12, 8))
for i in range(5): 
    plt.subplot(1, 5, i + 1)
    plt.imshow(X_train[25 + i])
    plt.title(y_train[25 + i])
plt.tight_layout()

In [ ]:
X_train = np.array(X_train[:42000])
X_val = np.array(X_val[:6000])
X_holdout = np.array(X_holdout[:12000])

y_train = np.array(y_train)
y_val = np.array(y_val)
y_holdout = np.array(y_holdout)

In [ ]:
assert len(X_train) == len(y_train)
assert len(X_val) == len(y_val)
assert len(X_holdout) == len(y_holdout)

In [ ]:
def normalize_float(x, is_3d_mri=True):
    """ 
    Function that performs max-division normalization on a `numpy.ndarray` 
    matrix. 
    """
    if is_3d_mri:
        assert(len(x.shape) >= 4)
    for i in range(x.shape[0]):
        x[i] /= np.max(x[i])
    return x


class IntensityRescale:
    """
    Rescale image itensities between 0 and 1 for a single image.
    Arguments:
        masked: applies normalization only on non-zero voxels. Default
            is True.
        on_gpu: speed up computation by using GPU. Requires torch.Tensor
             instead of np.array. Default is False.
    """

    def __init__(self, masked=True, on_gpu=False):
        self.masked = masked
        self.on_gpu = on_gpu

    def __call__(self, image):
        if self.masked:
            image = self.zero_masked_transform(image)
        else:
            image = self.apply_transform(image)

        return image

    def apply_transform(self, image):
        if self.on_gpu:
            return normalize_float_torch(image)
        else:
            return normalize_float(image, is_3d_mri = False)

    def zero_masked_transform(self, image):
        """ Only apply transform where input is not zero. """
        img_mask = image == 0
        # do transform
        image = self.apply_transform(image)
        image[img_mask] = 0.
        return image

def sagittal_flip(batch):
    """ 
        Expects shape (None, X, Y, Z, C).
        Flips along the X axis (sagittal).
        
    """
    thresh = 0.5
    batch_augmented = np.zeros_like(batch)
    for idx in range(len(batch)):
        rand = np.random.uniform()
        if rand > thresh:
            batch_augmented[idx] = np.flip(batch[idx], axis=0)
        else:
            batch_augmented[idx] = batch[idx]
    return batch_augmented

def translate(batch):
    """ 
        Expects shape (None, X, Y, Z, C).
        Translates the X axis.
    """
    batch_augmented = np.zeros_like(batch)
    for idx in range(len(batch)):
        rand = np.random.randint(-2, 3)
        if rand < 0:
            batch_augmented[idx,-rand:] = batch[idx,:rand]
        elif rand > 0:
            batch_augmented[idx,:-rand] = batch[idx,rand:]
        else:
            batch_augmented[idx] = batch[idx]
    return batch_augmented

In [ ]:
intensity = IntensityRescale(masked = False)

In [ ]:
class DataLoader(Sequence):
    def __init__(self, X, y, transform = None, batch_size = 32, shuffle = True, mask = None):
        self.X = X
        self.y = y 
        self.transform = transform 
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.mask = mask 
        
    def __len__(self):
        return int(np.ceil(len(self.y)/self.batch_size))
    
    def __getitem__(self, idx):
        image = np.expand_dims(self.X[idx * self.batch_size:(idx + 1) * self.batch_size], 3)
        label = np.array(self.y[idx * self.batch_size:(idx + 1) * self.batch_size], dtype=np.int8)
        
        if self.mask is not None: 
            for i in range(image.shape[0]):
                image[i] *= self.mask
        
        for transformation in self.transform: 
            image = transformation(image)
            
        return image, label
    
    def on_epoch_end(self):
        if self.shuffle:
            self.X, self.y = shuffle(self.X, self.y)

In [ ]:
train_loader = DataLoader(X = X_train, y = y_train, batch_size = 8, shuffle = True, transform = [intensity])

In [ ]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_loss') < LOSS_THRESHOLD):   
            print(f"\nReached below {LOSS_THRESHOLD} val loss, so stopping training!!")   
            self.model.stop_training = True

## Think about networks 

In [ ]:
def SimpleModel(input_shape, drop_rate = 0, weight_dcay = 0):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(2, activation='softmax'))
    return model

In [ ]:
lr = 5e-5
num_epochs = 125 # If using fully artificial data -> set to 75, sufficient for training
num_trials = 3  
batch_size = 128
store_models = True
patience = 10 
min_delta = 5e-4
LOSS_THRESHOLD = 0.05

In [ ]:
input_shape = X_train[0].shape

In [ ]:
accuracies = []
max_acc = []

In [ ]:
for i in range(num_trials):
    print('Trial %i' %i)
    model = SimpleModel(input_shape = (input_shape[0], input_shape[1], 1))
    opt = keras.optimizers.SGD(lr = lr, momentum = 0.9)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    earlystop = EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience, verbose=1, mode='auto')
    stop_training = myCallback()
    
    if store_models:
        result_path = os.path.join(result_dir, f"best_weights_trial_{i}.hdf5")
        model_checkpoint = ModelCheckpoint(result_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
        callbacks = [earlystop, model_checkpoint, stop_training]
    else:
        callbacks = [earlystop, stop_training]
        
    train_loader = DataLoader(X_train, y_train, transform=[intensity], batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(X_val, y_val, transform=[intensity], batch_size=32, shuffle=False)
    
    history = model.fit_generator(train_loader,
                        epochs=num_epochs,
                        validation_data=val_loader,
                        callbacks=callbacks)
    
    plt.figure(figsize=(11, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history["loss"], label = 'train')
    plt.plot(history.history["val_loss"], label = 'val')
    plt.legend()
    plt.title("Loss")
    plt.subplot(1, 2, 2)
    plt.plot(history.history["acc"], label = 'train')
    plt.plot(history.history["val_acc"], label = 'val')
    plt.legend()
    plt.title("Accuracy")
    plt.show()
    
    y_pred = model.predict_generator(val_loader)
    
    # Store results
    accuracies.append(history.history["val_acc"][-1])
    max_acc.append(np.max(history.history["val_acc"]))
    
training_time = time.time() - start_time 
print("Training Time: {}h:{}m:{}s".format(training_time//3600, (training_time//60)%60, training_time%60))

print("Validation final accuracies: \n {}".format(accuracies))
print("Validation final accuracies mean: {}".format(np.mean(accuracies)))
print("Validation best accuracies: \n {}".format(max_acc))
print("Validation best accuracies mean: {}".format(np.mean(max_acc)))


## Inference on model

In [ ]:
weights = ["best_weights_trial_0.hdf5", "best_weights_trial_1.hdf5", "best_weights_trial_2.hdf5"] 

In [ ]:
test_loader = DataLoader(X_holdout, y_holdout, transform=[intensity], batch_size=1, shuffle=False)

In [ ]:
accuracies = []

for fold, weight in enumerate(weights):
    print("Fold {}".format(fold))
    model = SimpleModel(input_shape = (input_shape[0], input_shape[1], 1))
    model_dir = os.path.join(result_dir, weight)
    model.load_weights(model_dir)
    
    opti = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=opti,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    # Evaluate
    res = model.evaluate_generator(test_loader)
    y_pred = model.predict_generator(test_loader)
  
    # Store results
    accuracies.append(res[1])

    # Print results
    print("Model accuracy {:.2f} %".format(res[1]*100))
    
print("######## Final results ########")
print("Accuracy mean {:.2f} %".format(np.mean(accuracies)*100))

In [ ]:
total_time = time.time() - start_time
print("Total time elapsed: {}h:{}m:{}s".format(
            total_time//3600, (total_time//60)%60, total_time%60))

In [ ]:
quit()